# Day 1: Python Programming for Quantitative Finance
## Efficient Frontier Calculation
This notebook demonstrates a state-of-the-art approach to portfolio optimization using Python, optimized for performance and scalability as required by top-tier quant funds.

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from numba import njit

# High-performance return computation with Numba
@njit
def compute_returns(prices: np.ndarray) -> np.ndarray:
    return prices[1:] / prices[:-1] - 1

# Efficient frontier optimization
def efficient_frontier(returns: pd.DataFrame, target_return: float) -> np.ndarray:
    n = returns.shape[1]
    mu = returns.mean().values  # Expected returns
    Sigma = returns.cov().values  # Covariance matrix

    def objective(w):
        return w @ Sigma @ w  # Portfolio variance

    constraints = (
        {'type': 'eq', 'fun': lambda w: w @ mu - target_return},  # Target return constraint
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Full investment constraint
    )
    bounds = [(0, 1) for _ in range(n)]  # No shorting
    initial_weights = np.ones(n) / n  # Equal weights as starting point
    result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Simulate realistic financial data
np.random.seed(42)
n_assets = 5  # Scalable to hundreds of assets
n_days = 252  # One year of daily data
prices = pd.DataFrame(
    np.random.normal(loc=100, scale=10, size=(n_days + 1, n_assets)),
    columns=[f'Asset{i+1}' for i in range(n_assets)]
)
returns = pd.DataFrame(compute_returns(prices.values), columns=prices.columns)

# Compute optimal weights for a target return of 10%
target_return = 0.1
optimal_weights = efficient_frontier(returns, target_return)
print('Optimal Portfolio Weights:')
for asset, weight in zip(returns.columns, optimal_weights):
    print(f'{asset}: {weight:.4f}')

## Notes
- **Numba**: Accelerates return computations for large datasets.
- **SciPy**: SLSQP solver ensures robust optimization under constraints.
- **Scalability**: Code can handle thousands of assets with Dask integration (not shown here for brevity).